## Data Wrangling - La cirugía de los datos

El **Data wrangling**, a veces denominada **data munging**, es el proceso de transformar y mapear datos de un dataset "en bruto" a otro formato con la intención de hacerlo más apropiado y valioso para una variedad de propósitos posteriores, como el análisis. Un **data wrangler** es una persona que realiza estas operaciones de transformación.

Esto puede incluir más munging, visualización de datos, agregación de datos, entrenamiento de un modelo estadístico, así como muchos otros usos potenciales. El data Wrangling como proceso generalmente sigue un conjunto de pasos generales que comienzan con la extracción de los datos en una forma sin procesar desde la fuente de datos, "*mezclando*" los datos sin procesar utilizando algoritmos (por ejemplo, clasificación) o analizando los datos en estructuras de datos predefinidas, y finalmente depositar el contenido resultante en un sistema de almacenamiento (o silos) para su almacenamiento y uso futuro.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
dspath = "../../datasets"
ccm = "customer-churn-model/Customer Churn Model.txt"

ccmpath = os.path.join(dspath, ccm)
data = pd.read_csv(ccmpath)

## Crear un subconjunto de datos. Filtrado de columnas

In [3]:
subset = data[["Account Length", "Phone", "Day Calls", "Day Mins" ]]
subset.describe()

,Account Length,Day Calls,Day Mins
count,3333.000000,3333.000000,3333.000000
mean,101.064806,100.435644,179.775098
std,39.822106,20.069084,54.467389
min,1.000000,0.000000,0.000000
25%,74.000000,87.000000,143.700000
50%,101.000000,101.000000,179.400000
75%,127.000000,114.000000,216.400000
max,243.000000,165.000000,350.800000


In [4]:
desired_columns = ["Account Length", "Phone", "Day Calls", "Night Calls" ]
all_columns = data.columns.values.tolist()
sublist = [x for x in all_columns if x not in desired_columns]
print (desired_columns)
print (all_columns)
print (sublist)

# alternativamente (aunque lo desordena)
a = set(desired_columns)
b = set(all_columns)
sublist = b-a
sublist = list(sublist)
print()
print ("***** otra sublist *****")
print()
print (sublist)

['Account Length', 'Phone', 'Day Calls', 'Night Calls']
['State', 'Account Length', 'Area Code', 'Phone', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?']
['State', 'Area Code', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?']

***** otra sublist *****

['Intl Calls', 'Eve Calls', 'Night Charge', 'State', 'VMail Message', 'Day Charge', 'Eve Charge', 'VMail Plan', 'Day Mins', 'Intl Mins', 'Churn?', "Int'l Plan", 'Intl Charge', 'Eve Mins', 'CustServ Calls', 'Area Code', 'Night Mins']


## Crear un subconjunto de datos.  filtrado de filas
### Usuarios de mas de 200 minutos

In [5]:
data[data["Day Mins"]>200].head(5)
data[data["Day Mins"]>200].shape

(1186, 21)

### Usuarios del área de NY

In [6]:
data[data["State"] == "NY"].head(5)
data[data["State"] == "NY"].shape

(83, 21)

### Filtros Booleanos
- AND -> & 
- OR -> |


In [7]:
data[(data["Day Mins"]>300) & (data["State"] == "NY")]

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
15,NY,161,415,351-7269,no,no,0,332.9,67,56.59,...,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True.
985,NY,64,415,345-9140,yes,no,0,346.8,55,58.96,...,79,21.21,275.4,102,12.39,13.3,9,3.59,1,True.


In [8]:
data[(data["Night Mins"] > data["Day Mins"])].shape

(2051, 21)

In [9]:
## minutos de día de noche y long de cuenta de los primeros 50 individuos

subset_first_50 = data[["State", "Area Code", "Phone", "Day Mins", "Night Mins", "Account Length"]][0:50]
subset_first_50.shape

(50, 6)

In [10]:
## Estado,, area y telf de la gente con mas de 330 min dia
subset_more_than_330 = data[["State", "Area Code", "Phone", "Day Mins"]][data["Day Mins"]>330]
subset_more_than_330

,State,Area Code,Phone,Day Mins
15,NY,415,351-7269,332.9
156,OH,415,370-9116,337.4
365,CO,415,343-5709,350.8
605,MO,415,373-2053,335.5
975,DE,510,332-6181,334.3
985,NY,415,345-9140,346.8
2594,OH,510,348-1163,345.3


### Selección por indice

In [11]:
subset_first_50.iloc[0:3, [2,3]]

,Phone,Day Mins
0,382-4657,265.1
1,371-7191,161.6
2,358-1921,243.4


#### Selección por índice y etiqueta

In [12]:
subset_first_50.loc[0:3, ["State","Day Mins"]]

,State,Day Mins
0,KS,265.1
1,OH,161.6
2,NJ,243.4
3,OH,299.4


### Creación de columnas calculadas
Se incluyen automáticamente al final del dataframe

In [13]:
subset_first_50["Total mins"] = subset_first_50["Day Mins"] + subset_first_50["Night Mins"]
subset_first_50.head()

,State,Area Code,Phone,Day Mins,Night Mins,Account Length,Total mins
0,KS,415,382-4657,265.1,244.7,128,509.8
1,OH,415,371-7191,161.6,254.4,107,416.0
2,NJ,415,358-1921,243.4,162.6,137,406.0
3,OH,408,375-9999,299.4,196.9,84,496.3
4,OK,415,330-6626,166.7,186.9,75,353.6
